In [1]:
# http://localhost:8080/notebooks/git/product-category/notebooks/prep_20210302A1.ipynb
prfx_prp = 'prep_20210302B1'

In [2]:
import pandas as pd
import numpy as np
from collections import Counter


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier




In [3]:
MIN_CNT = 50

In [4]:
%%time
df = pd.read_csv(f'../data/data__{prfx_prp}.csv')

<string>:2: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 1min 58s, sys: 16 s, total: 2min 14s
Wall time: 3min 1s


In [5]:
%%time
def try2eval(x):
    try:
        return eval(x)
    except SyntaxError:
        return []

# df = pd.read_csv(f'../data/data_sample__{prfx_prp}.csv')
df.fillna("", inplace=True)
df = df.astype(str)
df['category'] = df.category.apply(try2eval)
print("df.shape:", df.shape)
df['txt'] = df.title + " " + df.brand + " " + df.description + " " + df.feature

df.shape: (15605705, 7)
CPU times: user 3min 59s, sys: 15.3 s, total: 4min 14s
Wall time: 4min 14s


In [6]:
dmn2cnt = Counter(df.domain.value_counts().to_dict())
i2dmn = sorted(dmn2cnt.keys())
dmn2i = {v:k for k,v in enumerate(i2dmn)}
cat2cnt = Counter([j for i in df.category for j in i])
i2cat = sorted(k for k,v in cat2cnt.items() if v>50)
cat2i = {v:k for k,v in enumerate(i2cat)}

len(i2dmn), len(i2cat)

(26, 23301)

## split train val

In [7]:
np.random.seed(101)
msk_val = np.random.rand(len(df))>0.85
dftrn = df[~msk_val]
dfval = df[msk_val]
dftrn.shape, dfval.shape

((13264290, 8), (2341415, 8))

## rf model

In [ ]:
%%time
vec = TfidfVectorizer(ngram_range=(1,2),
                      min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                      smooth_idf=1, sublinear_tf=1)
Xtrn = vec.fit_transform(dftrn.txt)
Xval = vec.transform(dfval.txt)

Xtrn.shape, Xval.shape

In [ ]:
df['cat'] = df.category.apply(lambda x: [cat2i[o]])

In [ ]:
ys = np.zeros((len(df), len(i2cat)))

for i,cats in enumerate(df.category):
    idx_pos = [cat2i[cat] for cat in cats if cat in cat2i]
    ys[i,idx_pos] = 1

ys_trn = ys[~msk_val]
ys_val = ys[msk_val]

In [ ]:
clf = RandomForestClassifier(n_estimators=5, 
                             min_samples_leaf=2,
                             max_features=0.5,
                             n_jobs=-1)

In [ ]:
%%time
clf.fit(Xtrn, ys_trn)

In [ ]:
clf.n_outputs_

In [ ]:
prds_trn = clf.predict(Xtrn)
prds_prob_trn = clf.predict_proba(Xtrn)

np.mean(prds_trn==ys_trn), accuracy_score(ys_trn, prds_trn)

In [ ]:
prds_val = clf.predict(Xval)
prds_prob_val = clf.predict_proba(Xval)

np.mean(prds_val==ys_val), accuracy_score(ys_val, prds_val)

In [ ]:
for j in range(ys_val.shape[1]):
    print(f"{accuracy_score(ys_val[:,j], prds_val[:,j]):.2f}", end='; ')

## demo

In [ ]:
i = np.random.choice(range(len(ys_val)))
dct = dict(dfval.iloc[i])
print(dct['txt'])
prd = np.where(prds_val[i])[0]
print("Truth:", sorted(dct['category']))
print("Prediction:", [i2cat[i] for i in prd])